# Description

This will be for the training data only. 

In [1]:
import os
import glob
import subprocess
import pandas as pd

from subprocess import check_output

import pickle
# from datetime import datetime
from datetime import datetime as dt
from time import time

import pdb

import datetime
import shutil
import sys
import glob
import os
from subprocess import check_output
import argparse

# Useful functions

In [2]:
def adillyofapickle(basepath,dic, name):
    datefmt = '%m-%d-%Y_%I-%M-%S'
    st = dt.fromtimestamp(time()).strftime(datefmt)
    if os.path.exists(os.path.join(basepath,'tmp')):
        print('already have tmp')
    else:
        os.makedirs(os.path.join(basepath,'tmp'))
    pickle.dump(dic, open(os.path.join(basepath,'tmp','%s_%s'%(name,st)), 'wb'), protocol=4)


In [3]:
def onetoughjar(path2dic):
    with open(path2dic, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)


In [4]:
def reg_view(bak, epi, lab):
    display = plotting.plot_anat(bak, title="%s"%lab, display_mode='mosaic')
    return(display.add_contours(epi))

In [5]:
def setup_onsets(onset_file,sub, ses, task_id, run_num):
    inputs = (sub, ses, task_id, run_num)
    water_cue_outfile = "%s_%s_task-%s_%s_h2O_cue.txt"%(inputs)
    water_outfile="%s_%s_task-%s_%s_h2O_delivery.txt"%(inputs)
    rinse_outfile="%s_%s_task-%s_%s_rinse.txt"%(inputs)
    tasty_cue_outfile="%s_%s_task-%s_%s_milkshake_cue.txt"%(inputs)
    tasty_outfile="%s_%s_task-%s_%s_milkshake_delivery.txt"%(inputs)
    pe_outfile="%s_%s_task-%s_%s_pe.txt"%(inputs)
    mtch_milk_outfile="%s_%s_task-%s_%s_matched_milkshake.txt"%(inputs)
    mtch_h2o_outfile="%s_%s_task-%s_%s_matched_h2O.txt"%(inputs)
    ant_milk_outfile="%s_%s_task-%s_%s_anticipation_milkshake.txt"%(inputs)

    
#     # set output filenames 
#     water_cue_outfile="sub-{}_ses-{}_task-{}_run-{}_h2O_cue.txt".format(sub_num, ses_num, task_id, run_num)
#     water_outfile="sub-{}_ses-{}_task-{}_run-{}_h2O_delivery.txt".format(sub_num, ses_num, task_id, run_num)
#     rinse_outfile="sub-{}_ses-{}_task-{}_run-{}_rinse.txt".format(sub_num, ses_num, task_id, run_num)
#     tasty_cue_outfile="sub-{}_ses-{}_task-{}_run-{}_milkshake_cue.txt".format(sub_num, ses_num, task_id, run_num)
#     tasty_outfile="sub-{}_ses-{}_task-{}_run-{}_milkshake_delivery.txt".format(sub_num, ses_num, task_id, run_num)
#     pe_outfile="sub-{}_ses-{}_task-{}_run-{}_pe.txt".format(sub_num, ses_num, task_id, run_num)
#     mtch_milk_outfile="sub-{}_ses-{}_task-{}_run-{}_matched_milkshake.txt".format(sub_num, ses_num, task_id, run_num)
#     mtch_h2o_outfile="sub-{}_ses-{}_task-{}_run-{}_matched_h2O.txt".format(sub_num, ses_num, task_id, run_num)
#     ant_milk_outfile="sub-{}_ses-{}_task-{}_run-{}_anticipation_milkshake.txt".format(sub_num, ses_num, task_id, run_num)


    
    water_cue_dict= {}
    rinse_dict = {}
    tasty_cue_dict = {}
    tasty_dict = {}
    water_dict = {}
    pe_dict={}
    mtch_milk_dict = {}
    mtch_h2o_dict = {}
    ant_milk_dict = {}
    
    

    # load data
    file_df = pd.read_csv(onset_file, sep ="\t", header=None)
#     pdb.set_trace()

    #print("FILE: ", onset_file.split("/")[-1])
    start_time = file_df.loc[0, 0]
    #print("start time: ", start_time)

    file_df[0] = file_df[0] - start_time



    # view initial data
    #print(file_df[1].unique())
    #display(file_df.head(100))


    water_cue_df = file_df.loc[file_df[1] == 'Level image=waterlogo.jpg ' ].reset_index(drop=True)
    water_df = file_df.loc[file_df[1] == 'Level injecting via pump at address 0 ' ].reset_index(drop=True)
    rinse_df = file_df.loc[file_df[1] == 'Level RINSE ' ].reset_index(drop=True)
    tasty_cue_df = file_df.loc[file_df[1] == 'Level image=tasty.jpg ' ].reset_index(drop=True)
    tasty_df = file_df.loc[file_df[1] == 'Level injecting via pump at address 1 ' ].reset_index(drop=True)   
    


    # setting up PE scenarios:
    #print("PE df:")
    #print("before edits: \n") 
    #display(water_df)

    # case PE: cue is tasty, taste is water
    water_cue_indexes=[] # will hold the indexes of the rows we don't want 
    milk_cue_indexes=[]
    water_indexes=[] # will hold the indexes of the rows we don't want 
    milk_indexes=[]

    pe_h2o_df = file_df.loc[file_df[1] == 'Level injecting via pump at address 0 ' ]

    # get pe and matched h2o settings
    for i in pe_h2o_df.index.values:
        #print(i-2)                

        if "tasty" in file_df.loc[i-2, :][1]:
            #display(file_df.loc[i-2, :][1])
            milk_cue_indexes.append(i-2)
            milk_indexes.append(i)
        elif  'tasty' in file_df.loc[i-1, :][1]:
            #display(file_df.loc[i-1, :][1])
            milk_cue_indexes.append(i-1)
            milk_indexes.append(i)
        elif "water" in file_df.loc[i-2, :][1]:
            #display(file_df.loc[i-2, :][1])
            water_cue_indexes.append(i-2)
            water_indexes.append(i)
        elif  'water' in file_df.loc[i-1, :][1]:
            #display(file_df.loc[i-1, :][1])
            water_cue_indexes.append(i-1)
            water_indexes.append(i)
        else:
            pass


    # pe case - cue is for milkshake but they taste water        
    pe_df = pe_h2o_df.drop(water_indexes).reset_index(drop=True)
    #print("PE: ")
    #display(pe_df)

    # anticipation milk 
    #print("Anticipation milk: ")
    # anticipation milk case - cue from pes 
    anticipation_milk_df =  file_df.loc[milk_cue_indexes, :]
#     display(anticipation_milk_df)

    # case matched h2O 
    #print("Matched h2O: ")
    matched_h2o_df = pe_h2o_df.drop(milk_indexes).reset_index(drop=True)
    #display(matched_h2o_df)

    # case matched milk
    milk_mtch_cue_indexes = []
    milk_taste_df = file_df.loc[file_df[1] == 'Level injecting via pump at address 1 ' ]
    for i in milk_taste_df.index.values:
        if "tasty" in file_df.loc[i-2, :][1]:
            #display(file_df.loc[i-2, :][1])
            milk_mtch_cue_indexes.append(i-2)
        elif  'tasty' in file_df.loc[i-1, :][1]:
            #display(file_df.loc[i-1, :][1])
            milk_mtch_cue_indexes.append(i-1)
        else:
            pass

    #print('Matched milk: \n')
    matched_milk_df = file_df.loc[milk_mtch_cue_indexes]
    #display(matched_milk_df)
    new_water_cue_df = pd.DataFrame(water_cue_df[0])
    new_water_df = pd.DataFrame(water_df[0])
    
    new_tasty_cue_df = pd.DataFrame(tasty_cue_df[0])
    new_tasty_df = pd.DataFrame(tasty_df[0])
    new_pe_df = pd.DataFrame(pe_df[0])
    new_matched_milk_df = pd.DataFrame(matched_milk_df[0])
    new_matched_h2o_df= pd.DataFrame(matched_h2o_df[0])
    new_anticipation_milk_df = pd.DataFrame(anticipation_milk_df[0])
    
    print(rinse_df.shape) #(0, 3)
    if rinse_df.shape[0] == 0:
        x = pd.DataFrame(water_df[0])+7
        y = pd.DataFrame(tasty_df[0])+7
        z = pd.concat([x,y], ignore_index=True,sort=True)
        z = z.sort_values(0)
        new_rinse_df = pd.DataFrame(z)
    else:
        new_rinse_df = pd.DataFrame(rinse_df[0])
        
    # fill in durationa and modulus
    new_water_cue_df['dur'] = 1
    new_water_cue_df['mod'] = 1

    new_water_df['dur'] = 3
    new_water_df['mod'] = 1

    new_rinse_df['dur'] = 2
    new_rinse_df['mod'] = 1

    new_tasty_cue_df['dur'] = 1
    new_tasty_cue_df['mod'] = 1

    new_tasty_df['dur'] = 3
    new_tasty_df['mod'] = 1

    new_pe_df['dur'] = 3
    new_pe_df['mod'] = 1

    new_matched_milk_df['dur'] = 3
    new_matched_milk_df['mod'] = 1

    new_matched_h2o_df['dur'] = 3
    new_matched_h2o_df['mod'] = 1

    new_anticipation_milk_df['dur'] = 1
    new_anticipation_milk_df['mod'] = 1

    line="0\t0\t0\n"
    
    #print('writing files...')
    output_dir = '/projects/niblab/experiments/bromocriptine/data/onsets/tmp'
    
    
    if new_water_cue_df.empty:
        with open(os.path.join(output_dir,  water_cue_outfile ), 'w') as file:
            file.write(line)
    else:
        new_water_cue_df.to_csv(os.path.join(output_dir, water_cue_outfile ),sep="\t", header=False, index=False)

    if new_water_df.empty:
        with open(os.path.join(output_dir, water_outfile ), 'w') as file:
            file.write(line)
    else:
        new_water_df.to_csv(os.path.join(output_dir, water_outfile ),sep="\t", header=False, index=False)

    if new_rinse_df.empty:
        with open(os.path.join(output_dir, rinse_outfile ), 'w') as file:
            file.write(line)
    else:
        new_rinse_df.to_csv(os.path.join(output_dir, rinse_outfile),sep="\t", header=False, index=False)

    if new_tasty_cue_df.empty:
        with open(os.path.join(output_dir, tasty_cue_outfile), 'w') as file:
            file.write(line)
    else:
        new_tasty_cue_df.to_csv(os.path.join(output_dir, tasty_cue_outfile ),sep="\t", header=False, index=False)

    if new_tasty_df.empty:
        with open(os.path.join(output_dir, tasty_outfile), 'w') as file:
            file.write(line)
    else:
        new_tasty_df.to_csv(os.path.join(output_dir, tasty_outfile ),sep="\t", header=False, index=False)

    if new_pe_df.empty:
        with open(os.path.join(output_dir, pe_outfile), 'w') as file:
            file.write(line)
    else:
        new_pe_df.to_csv(os.path.join(output_dir, pe_outfile ),sep="\t", header=False, index=False)

    if new_matched_milk_df.empty:
        with open(os.path.join(output_dir, mtch_milk_outfile), 'w') as file:
            file.write(line)
    else:
        new_matched_milk_df.to_csv(os.path.join(output_dir, mtch_milk_outfile ),sep="\t", header=False, index=False)

    if new_anticipation_milk_df.empty:
        with open(os.path.join(output_dir, ant_milk_outfile), 'w') as file:
            file.write(line)
    else:
        new_anticipation_milk_df.to_csv(os.path.join(output_dir, ant_milk_outfile ),sep="\t", header=False, index=False)

    if new_matched_h2o_df.empty:
        with open(os.path.join(output_dir, mtch_h2o_outfile), 'w') as file:
            file.write(line)
    else:
        new_matched_h2o_df.to_csv(os.path.join(output_dir, mtch_h2o_outfile ),sep="\t", header=False, index=False)

    print("_______________________________________________")



## Paths

In [6]:
script_path = '/projects/niblab/scripts/BRO/'
fmriprep_path = '/projects/niblab/experiments/bromocriptine/fmriprep'
out_path = '/projects/niblab/data/BRO'
datadir = '/projects/niblab/experiments/bromocriptine/data/preprocessed/'


# ROIs

## Featquery
https://fsl.fmrib.ox.ac.uk/fslcourse/graduate/lectures/practicals/feat1/
Featquery allows you to calculate certain statistics, either at a voxel of interest, or averaged over a region of interest using a mask. We will use the standard-space mask that we created earlier. Start up Featquery from the terminal:

Featquery &
(or Featquery_gui & if you are on a Mac).

Select the .feat directory created in your first analysis on the fmri dataset, called fmri.feat unless another output name was specified. If your FEAT has still not finished running, you can use another subject that we have already run for you in ~/fsl_course_data/feat1/prebaked

Featquery automatically reads the FEAT directory and gives you the appropriate options as to which statistics you can choose to investigate.

Select the following statistics for the contrast that looks at activation for novel (i.e. the [1 0] contrast), the contrast that looks at activation for familiar (i.e. the [0 1] contrast) and the contrast that looks at activation for novel images over and above familiar (i.e. the [1 -1] contrast):
stats/cope (unthresholded contrast of parameter estimate)
stats/tstat (unthresholded t statistics)
stats/zstat (unthresholded z statistics)
thresh_zstat (thresholded z statistics)

Note that the numbers you need to select will depend on the order you specified your contrasts earlier.

In the Input ROI selection panel, enter the mask that you created earlier (i.e. right_hippocampus_mask.nii.gz) as the Mask Image (note - Featquery can take either a standard-space mask OR a lowres one in the original dataspace OR a mask in the space of the structural image)
In the Output options section, select the Convert PE/COPE values to % option
Select the Do not binarise mask (allow weighting) option
Press Go and a web browser showing the estimated statistics should popup shortly (possibly a minute or two if things are busy).

# fmriprep

the issue with the command (i think) is it wants all the flags before the positional arguments, and i needed write permission on the output directory (see new directory) and I needed to use the exec not run argument in the slurm command. 

```
fmriprep -v --participant-label sub-018 --skip_bids_validation --fs-license-file /base_dir/freesurfer/license.txt --fs-no-reconall --longitudinal --output-spaces MNI152NLin2009cAsym --omp-nthreads 16 --n_cpus 16 --ignore slicetiming --bold2t1w-dof 12 --fd-spike-threshold 0.9 -w /base_dir/experiments/bromocriptine/data/fmriprep/fp_wf --resource-monitor --stop-on-first-crash /base_dir/experiments/bromocriptine/data/bids/bids /base_dir/data/BRO participant
```

# Keeping track of the cleaning

In [ ]:
cleaning_summary = {}
for subs in glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/preprocessed','sub-*')):
    sub = subs.split('/')[-1]
    cleaning_summary[sub]=[]

In [ ]:
cols = ['ses','run','status']
for sub, listi in cleaning_summary.items():
    x = os.path.join('/projects/niblab/data/BRO/level1','%s_ses-*_training_run-*.feat'%sub)
    ses = []
    run = []
    status = []
    for j in glob.glob(x):
        if os.path.exists(j):
            k = j.split('/')[-1].split('_')
            ses.append(k[1]) 
            run.append(k[-1].split('.')[0]) 
            status.append('complete')
    dicti = {'ses': ses, 'run': run, 'status': status}   
    df = pd.DataFrame(dicti)
    cleaning_summary[sub].append(df)            

## Find which subjects have full data

In [39]:
cvs = {}
for dat in glob.glob(os.path.join(script_path,'renci_link','*.csv')):
    typ = dat.split('_')[-1].split('.')[0]
    print(typ)
    cvs[typ] = pd.read_csv(dat,names=['%s'%typ], dtype = 'object')
    

training2
PE1
PE2
anat
training1


In [40]:
df1 = pd.merge(cvs['training1'], cvs['training2'], left_on='training1', right_on='training2')
df = pd.merge(df1, cvs['anat'], left_on='training1', right_on='anat')
df.to_csv(os.path.join(out_path,'complete_subs.csv'), index=False, header = False)  

In [ ]:
subs = df['training1']

In [ ]:
df.to_csv(os.path.join(out_path,'complete_subs.csv'), index=False, header = False)  

In [ ]:
data_dict = {}

for subpath in glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/bids/bids','sub-*')):
    sub = subpath.split('/')[-1]
    data_dict[sub] = {'bids':
                  {'ses-1':{'func':{'path':[],'missing':[]},'anat':{'path':[],'missing':[]}},
                   'ses-2':{'func':{'path':[],'missing':[]},'anat':{}}},
                  'fmriprep':
                  {'ses-1': {'func':{'path':[],'missing':[]},'anat':{'path':[],'missing':[]}},
                   'ses-2':{'func':{'path':[],'missing':[]},'anat':{'path':[],'missing':[]}}},
                  'prepro':{'ses-1':{'func':{'path':[],'missing':[]},'anat':{'path':[],'missing':[]}},'ses-2':{'func':{'path':[],'missing':[]},'anat':{'path':[],'missing':[]}}}
                     }






In [ ]:
base_dir = '/projects/niblab/'
path_dict = {'bids':os.path.join(base_dir,'experiments/bromocriptine/data/bids/bids'),
            'fmriprep':[os.path.join(base_dir,'experiments/bromocriptine/data/fmriprep'), os.path.join(base_dir,'data/BRO/fmriprep')],
            'prepro':os.path.join(base_dir,'experiments/bromocriptine/data/preprocessed')}
deets = {'ses-1':['run-1','run-2'], 'ses-2':['run-1','run-2']}


In [ ]:
def extractor(nifti, output):
    bet_cmd = 'bet %s %s -F'%(nifti, output)
    subprocess.run(['bet','%s'%nifti,'%s'%output,'-F'])

In [ ]:
test = {}
for subs in glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/preprocessed','sub-*')):
    sub = subs.split('/')[-1]
    test[sub] = {}
    for ses, runs in deets.items():
        test[sub][ses] = {}
        for run in runs:
            test[sub][ses][run] = []
            fil = '%s_%s_task-training_%s_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz'%(sub,ses,run)
            if os.path.exists(os.path.join(subs,ses,'func','niftis',fil)):
#                 print('true')
                test[sub][ses][run].append('prepro exists')
                moco=glob.glob(os.path.join(subs,ses,'func','motion_parameters','%s_%s_task-training_%s_moco*.txt'%(sub, ses, run)))
                if len(moco) == 6:
                    test[sub][ses][run].append('got mocos')
            else:
#                 print('false')
                fil2 = '%s_%s_task-training_%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'%(sub,ses,run)
                pospath = os.path.join('/projects/niblab/experiments/bromocriptine/data/fmriprep',sub,ses,'func',fil2)
                pospath2 = os.path.join('/projects/niblab/data/BRO/fmriprep',sub,ses,'func',fil2)
                ouut = os.path.join(subs,ses,'func','niftis',fil)
                if os.path.exists(pospath):
                    print('fmriprp')
                    test[sub][ses][run].append('needs bet')
#                     print(pospath)
#                     print(ouut)
                    extractor(pospath,ouut)
                elif os.path.exists(pospath2):
                    test[sub][ses][run].append('needs bet new fmriprep')
                    print('%s newly minted'%sub)
                    extractor(pospath2,ouut)
                else:
                    fil4 = '%s_%s_task-training_%s_bold.nii.gz'%(sub,ses,run)
                    pospath3 = os.path.join('/projects/niblab/experiments/bromocriptine/data/bids/bids',sub,ses,'func',fil4)
                    if os.path.exists(pospath3):
                        print('%s needs fmriprep'%sub)
                        test[sub][ses][run].append('%s needs fmriprep'%sub)
                    

adillyofapickle('/projects/niblab/scripts/BRO/NOTES',test, 'data_check')     




## Re ran fmriprep on
'sub-004'<- complete  
'sub-006'<- fails fieldmap    
'sub-008'<- complete   
'sub-014'<- complete  
'sub-015'<- complete     
'sub-018'<- complete   
'sub-021'<- complete    
'sub-031'<- failed fieldmap but different this time    
'sub-034'<- complete     
'sub-048'<- damaged file    
'sub-049'<- complete    
'sub-050'<- complete    
'sub-054'<- fail    
'sub-055'<- totally missing    
'sub-056'<- complete    

In [ ]:
nifts = {'complete':[], 'missing':{'scans':[],'moco':[]}}
for x in glob.glob(os.path.join(datadir, 'sub-*','ses-*','func')):
    nifs = glob.glob(os.path.join(x,'niftis','sub-*_ses-*_task-*_run-*_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz'))
    moco = glob.glob(os.path.join(x,'motion_parameters','sub-*_ses-*_task-*_run-*_moco*.txt'))
    if len(nifs) > 0 and len(moco) > 0: 
        nifts['complete'].append(x)
    elif len(nifs) > 0 and len(moco) == 0:
        nifts['missing']['moco'].append(x)
    elif len(nifs) == 0 and len(moco) == 0:
        nifts['missing']['scans'].append(x)
    

In [ ]:
adillyofapickle('/projects/niblab/scripts/BRO/NOTES',nifts, 'missing_summary')     

In [ ]:
dcms = {'has':[],'hasnot':[],'needs_fmriprep':[]}
for x in nifts['missing']['scans']:
#     print(x)
    sub = x.split('/')[-3]
    ses = x.split('/')[-2]
    t = os.path.join('/projects/niblab/experiments/bromocriptine/data/bids/bids/','%s'%sub,'%s'%ses,'func')
    for y in glob.glob(os.path.join(t,'%s_%s_task-training_run-*_bold.nii.gz'%(sub,ses))):
        if os.path.exists(y):
            print('%s'%sub)
            for y in glob.glob(os.path.join(t,'%s_%s_task-training_run-*_bold.nii.gz'%(sub,ses))):
                fil = y.split('/')[-1].split('.')[0]
                output = os.path.join(datadir, '%s/%s/func/niftis/%s_brain.nii.gz'%(sub,ses, fil))
                dcms['needs_fmriprep'].append(y)
    else:
        dcm = os.path.join('/projects/niblab/experiments/bromocriptine/data/bids/bids/sourcedata',ses)
        if os.path.exists(dcm):
            if any(os.scandir(dcm)):
                print('not empty')
                dcms['has'].append(dcm)
            else:
                print('totally missing')
                dcms['hasnot'].append(sub)
                dcms['hasnot'].append(ses)


In [ ]:
adillyofapickle('/projects/niblab/scripts/BRO/NOTES',dcms, 'missingprep_summary')     

# Motion correction

# Cleaning summary
| sub          | ses         |  run      | reason     |
| -----------  | ----------- |-----------|----------- |
|sub-001 |1 |1|FAIL|
|---------------------------|
| sub-004      | ses-1       |  run-1    | FD fail|
| sub-004      | ses-2       |  run-1    | FD fail|
|---------------------------|
| sub-006      | all       |  all    | fmriprep fail    |
|---------------------------|
|sub-018|2 |2| MILKSHAKE_CUE|       
|sub-018|1|2| FD fail    |
|sub-018|2|1| FD fail    |
|sub-018|2|1| FD fail    |
|---------------------------|
|sub-020 |2 |1|milkshake_cue.txt fail|
|sub-020 |2 |2|Can't open MILKSHAKE_CUE |
|sub-020 |1 |2| DOESNT EXIST|
|---------------------------|
|sub-023 |2 |1    | MILKSHAKE_CUE|
|sub-023 |2 |2    | wrong onset|
|---------------------------|
|sub-027|1|2| CHECK|
|---------------------------|
|sub-031|all|  all    | fmriprep fail    |
|---------------------------|
|sub-033|1|1|just missing need to find|
|sub-033|1|2|just missing need to find|
|---------------------------|
|sub-035|2|1|MILKSHAKE_CUE |
|sub-035|2|2|MILKSHAKE_CUE|
|---------------------------|
|sub-037 |1 |2|missing onset|  
|sub-037 |2 |1|missing onset|  
|sub-037 |2 |2|missing onset| 
|---------------------------|
|sub-041 |2|1| wrong onset|
|sub-042 |all|all| bad onsets|
|---------------------------|
|sub-046 |1|1|just missing need to find|
|sub-046 |1|2|just missing need to find|
|sub-046 |2|2|just missing need to find|
|---------------------------|
|sub-047 |1|2|just missing need to find|
|sub-047 |2|1|just missing need to find|
|sub-047 |2|2|just missing need to find|
|---------------------------|
|sub-048 |all|all| fmriprep fail    |
|sub-049 |1|1| FD fail    |
|---------------------------|
|~sub-050~ |~1~ |~2~| ~moco0.txt~ |
|sub-050 |2 |1| just missing need to find|
|sub-050 |2 |2| just missing need to find|
|---------------------------|
|sub-053 |1 |2| just missing need to find|
|sub-053 |2 |1| just missing need to find|
|sub-053 |2 |2| just missing need to find|
|---------------------------|
|sub-054 | all|  all    | fmriprep fail    |
|---------------------------|
|sub-055 | all|  all    | fmriprep fail    |
|---------------------------|
|sub-056 |1|1| just missing need to find|
|sub-056 |1|2| just missing need to find|
|sub-056 |2|1| FD fail    |
|sub-056 |2|2| FD fail    |  



In [ ]:
test = pd.read_csv('/projects/niblab/data/BRO/fmriprep/sub-008/ses-2/func/sub-008_ses-2_task-training_run-1_desc-confounds_regressors.tsv', sep ='\t')




#sub-011_ses-1_task-resting_moco0.txt




In [ ]:
interest = ['trans_x','trans_y', 'trans_z','rot_x','rot_y','rot_z']
mco = ['%s_%s_task-training_%s_moco0.txt','%s_%s_task-training_%s_moco1.txt','%s_%s_task-training_%s_moco2.txt',
      '%s_%s_task-training_%s_moco3.txt','%s_%s_task-training_%s_moco4.txt','%s_%s_task-training_%s_moco5.txt']






## By hand moco fix

In [ ]:
sub = 'sub-050'
ses = 'ses-1'
fil = '%s_%s_task-training_run-*_desc-confounds_regressors.tsv'%(sub,ses)
pospath = os.path.join('/projects/niblab/experiments/bromocriptine/data/fmriprep/',sub,ses,'func',fil)
pospath2 = os.path.join('/projects/niblab/data/BRO/fmriprep',sub,ses,'func',fil)
x = glob.glob(pospath2)
x

In [ ]:
df = pd.read_csv(x[1], sep = '\t')
run = x[1].split('/')[-1].split('_')[-3]
for i,x in enumerate(interest):
    path = '/projects/niblab/experiments/bromocriptine/data/preprocessed/%s/%s/func/motion_parameters'%(sub,ses)
    npath = os.path.join(path,mco[i]%(sub,ses,run))
    print(npath)
    df[x].to_csv(npath, sep = ' ', index = False, header = False) 

In [ ]:
for x in regress2:
    run = x.split('/')[-1].split('_')[-3]
    df = pd.read_csv(x, sep = '\t')
    for i,x in enumerate(interest):
        path = '/projects/niblab/experiments/bromocriptine/data/preprocessed/%s/%s/func/motion_parameters'%(sub,ses)
        npath = os.path.join(path,mco[i]%(sub,ses,run))
        print(npath)
        df[x].to_csv(npath, sep = ' ', index = False, header = False) 

## Loop moco fix

In [ ]:
for nif in nifts['missing']['moco']:
    sub = nif.split('/')[-3]
    ses = nif.split('/')[-2]
    if os.path.exists(os.path.join(nif,'motion_parameters')):
        fil = '%s_%s_task-training_run-*_desc-confounds_regressors.tsv'%(sub,ses)
        pospath = os.path.join('/projects/niblab/experiments/bromocriptine/data/fmriprep/',sub,ses,'func',fil)
        pospath2 = os.path.join('/projects/niblab/data/BRO/fmriprep',sub,ses,'func',fil)
        regress = glob.glob(pospath)
        regress2 = glob.glob(pospath2)
        if len(regress) == 2:
            print('got the tsv')
            for x in regress:
                print(x)
                df = pd.read_csv(x, sep = '\t')
        elif len(regress2) == 2:
            print('got the new tsv')
            for x in regress2:
                run = x.split('/')[-1].split('_')[-3]
                df = pd.read_csv(x, sep = '\t')
                for i,x in enumerate(interest):
                    path = '/projects/niblab/experiments/bromocriptine/data/preprocessed/%s/%s/func/motion_parameters'%(sub,ses)
                    npath = os.path.join(path,mco[i]%(sub,ses,run))
                    print(npath)
                    df[x].to_csv(npath, sep = ' ', index = False, header = False)   
    else:
        print('need dir')
        os.mkdir(os.path.join(nif,'motion_parameters')) 

# Confounds 
Many subjects are missing their confounds. Need to make new ones

In [ ]:
import numpy as np

In [ ]:
fail_fd = {}
for tsv in glob.glob('/projects/niblab/data/BRO/fmriprep/sub-*/ses-*/func/*.tsv'):
    sub = tsv.split('/')[-4]
    ses = tsv.split('/')[-3]
    task = tsv.split('/')[-1].split('_')[2]
    if task == 'task-resting':
        run = 'run-00'
    else:
        run = tsv.split('/')[-1].split('_')[3]
    df = pd.read_csv(tsv, sep = '\t')
    x = df.columns.str.contains(r'motion_outlier')
    fd = df.iloc[:, x]
    if fd.shape[1] == 0:
        print('no fd')
        d = pd.DataFrame(np.zeros((243, 1)))
        d.write_csv(os.path.join(datadir,sub,ses,'func','confounds','%s_%s_%s_%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt'%(sub,ses,task,run)),
                         sep = ' ', index = False, header = False)
    else:
        fd2 = fd.sum(axis = 1)
        if fd2.sum(axis = 0) > 61:
            fail_fd[tsv] = fd2.sum(axis = 0)
        else:
            print(sub)
            print(run)
            df = pd.DataFrame(fd2)
            fd2.to_csv(os.path.join(datadir,sub,ses,'func','confounds','%s_%s_%s_%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt'%(sub,ses,task,run)),
                         sep = ' ', index = False, header = False)
            
            
            
            
            
            
            

In [ ]:
adillyofapickle('/projects/niblab/scripts/BRO/NOTES',fail_fd, 'df_fail')     


# Level 1

## Need these paths
- output
- functional
- confound
- anat
- milkshake_cue_file
- milkshake_delivery_file
- h2O_cue_file
- h2O_delivery_file
- rinse_file
- moco0_file
- moco1_file
- moco2_file
- moco3_file
- moco4_file
- moco5_file


- sub-036_ses-2_task-training_run-1_h2O_cue.txt 
- sub-036_ses-2_task-training_run-1_h2O_delivery.txt 
- sub-036_ses-2_task-training_run-1_milkshake_cue.txt
- sub-036_ses-2_task-training_run-1_milkshake_delivery.txt
- sub-036_ses-2_task-training_run-1_rinse.txt  
#########
- sub-013_ses-2_task-pe_run-1_anticipation_milkshake.txt
- sub-013_ses-2_task-pe_run-1_matched_h2O.txt
- sub-013_ses-2_task-pe_run-1_matched_milkshake.txt
- sub-013_ses-2_task-pe_run-1_pe.txt
- sub-013_ses-2_task-pe_run-1_rinse.txt


## Onsets
from bro_onsets.ipynb

# Cleaning summary
| sub          | ses         |  run      | reason     |
| -----------  | ----------- |-----------|----------- |
|sub-001 |||COMPLETE FAIL|
|---------------------------|
| sub-004      | ses-1       |  run-1    | FD fail|
| sub-004      | ses-2       |  run-1    | FD fail|
|---------------------------|
| sub-006||| COMPLETE FAIL|       
|---------------------------|
|sub-018||| COMPLETE FAIL|       
|---------------------------|
|sub-020 |2 |1|COMPLETE FAIL|
|---------------------------|
|sub-023 |2 |1    | ~MILKSHAKE_CUE~, found need to check|
|sub-023 |2 |2    | ~wrong onset~, found need to check|
|---------------------------|
|sub-027|1|2| CHECK|
|---------------------------|
|sub-031||| COMPLETE FAIL|       
|---------------------------|
|sub-033|1|1|just missing need to find|
|sub-033|1|2|just missing need to find|
|---------------------------|
| sub-025||| COMPLETE FAIL (no ses-2)|
|---------------------------|
|sub-037 |1 |2|missing onset|  
|sub-037 |2 |1|missing onset|  
|sub-037 |2 |2|missing onset| 
|---------------------------|
|sub-041 |2|1| wrong onset|
|sub-042 |all|all| bad onsets|
|---------------------------|
|sub-046 |1|1|just missing need to find|
|sub-046 |1|2|just missing need to find|
|sub-046 |2|2|just missing need to find|
|---------------------------|
|sub-047 |1|2|just missing need to find|
|sub-047 |2|1|just missing need to find|
|sub-047 |2|2|just missing need to find|
|---------------------------|
|sub-048 ||| COMPLETE FAIL|       
|sub-049 |1|1| FD fail    |
|---------------------------|
|~sub-050~ |~1~ |~2~| ~moco0.txt~ |
|sub-050 |2 |1| just missing need to find|
|sub-050 |2 |2| just missing need to find|
|---------------------------|
|sub-053 |1 |2| just missing need to find|
|sub-053 |2 |1| just missing need to find|
|sub-053 |2 |2| just missing need to find|
|---------------------------|
|sub-054||| COMPLETE FAIL|       
|---------------------------|
|sub-055||| COMPLETE FAIL|       
|---------------------------|
|sub-055||| COMPLETE FAIL, failed both fd checks in ses-2|  

In [ ]:
onset_path = '/projects/niblab/experiments/bromocriptine/data/onsets/output_onsets'
onset_path2 = '/projects/niblab/experiments/bromocriptine/data/onsets/tmp'

In [ ]:
subject = '/projects/niblab/experiments/bromocriptine/data/preprocessed/sub-*'
subjects = glob.glob(subject)

In [ ]:
onset_compl = {}
sess = ['1','2']
runs = ['1','2']
tasks = ['training','pe']
tastes_train = ['h2O_cue','milkshake_cue','h2O_delivery','milkshake_delivery','rinse']
tastes_pe = ['anticipation_milkshake','matched_h2O','matched_milkshake','pe','rinse']
# sub-007_ses-2_task-pe_1_anticipation_milkshake.txt
for subs in subjects:
    sub = subs.split('/')[-1]
    onset_compl[sub] = {}
    print(sub)
    for ses in sess:
        print(ses)
        onset_compl[sub]['ses-%s'%ses] = {}
        for run in runs:
            print(run)
            onset_compl[sub]['ses-%s'%ses]['run-%s'%run] = {'training': {'exists': [], 'missing':[]}, 'pe': {'exists': [], 'missing':[]}}
            for task in tasks:
                if task == 'training':
                    for taste in tastes_train:
                        x = os.path.join(onset_path,'%s_ses-%s_task-%s_run-%s_%s.txt'%(sub,ses,task,run,taste))
                        x2 = os.path.join(onset_path2,'%s_ses-%s_task-%s_run-%s_%s.txt'%(sub,ses,task,run,taste))
                        if os.path.exists(x):
                            onset_compl[sub]['ses-%s'%ses]['run-%s'%run]['training']['exists'].append(x)
                        elif os.path.exists(x2):
                            onset_compl[sub]['ses-%s'%ses]['run-%s'%run]['training']['exists'].append(x)
                        else:
                            y = 'missing %s_ses-%s_task-%s_run-%s_%s.txt'%(sub,ses,task,run,taste)
                            onset_compl[sub]['ses-%s'%ses]['run-%s'%run]['training']['missing'].append(y)
                else:
                    for taste in tastes_pe:
                        x = os.path.join(onset_path,'%s_ses-%s_task-%s_run-%s_%s.txt'%(sub,ses,task,run,taste))
                        if os.path.exists(x):
                            onset_compl[sub]['ses-%s'%ses]['run-%s'%run]['pe']['exists'].append(x)
                        else:
                            y = 'missing %s_ses-%s_task-%s_run-%s_%s.txt'%(sub,ses,task,run,taste)
                            onset_compl[sub]['ses-%s'%ses]['run-%s'%run]['pe']['missing'].append(y)
                        
                        
                        
                        
                        
                        
                        

In [ ]:
missing_ons = {}
for sub, data in onset_compl.items():
    print(sub)
    missing_ons[sub] = {}
    for ses, dat in data.items():
        print(ses)
        missing_ons[sub][ses] = []
        for run, da in dat.items():
            print(len(da))
            if len(da['training']['missing'])>0:
                print('%s %s %s is missing'%(sub,ses,run))
                missing_ons[sub][ses].append(run)
adillyofapickle('/projects/niblab/scripts/BRO/NOTES',missing_ons, 'missing_ons')     


In [ ]:
msms = {}
for sub, data in missing_ons.items():
    for ses, dat in data.items():
        if len(dat) > 0:
            msms[sub] = data
            continue
        

In [ ]:
msms

# Onsets by hand

In [ ]:
subs = ['sub-007',
 'sub-008',
 'sub-009',
 'sub-010',
 'sub-011',
 'sub-012',
 'sub-014',
 'sub-015',
 'sub-016',
 'sub-017',
 'sub-018']

In [34]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_018_session02_run02_2018-04-24-09_00_36_subdata.log'
sub = 'sub-018'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(32, 3)
_______________________________________________


In [30]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_017_session02_run02_2018-04-23-19_36_50_subdata.log'
sub = 'sub-017'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(32, 3)
_______________________________________________


In [26]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_016_session02_2017-12-18-19_32_06_subdata.log'
sub = 'sub-016'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(0, 3)
_______________________________________________


In [22]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_015_session02_2017-12-22-14_42_20_subdata.log'
sub = 'sub-015'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(0, 3)
_______________________________________________


In [18]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_014_session02_2017-12-21-10_58_56_subdata.log'
sub = 'sub-014'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(0, 3)
_______________________________________________


In [14]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_012_session02_2017-11-20-08_14_18_subdata.log'
sub = 'sub-012'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

(0, 3)
_______________________________________________


In [10]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_011_session02_2017-11-21-08_15_00_subdata.log'
sub = 'sub-011'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


(0, 3)
_______________________________________________


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_010_session02_2017-12-02-13_12_40_subdata.log'
sub = 'sub-010'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_009_session02_2017-12-02-09_51_45_subdata.log'
sub = 'sub-009'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_008_session02_2017-11-20-19_07_16_subdata.log'
sub = 'sub-008'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_007_session02_2018-03-24-15_38_39_subdata.log'
sub = 'sub-007'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_004_session02_run02_2018-04-06-09_35_41_subdata.log'
sub = 'sub-004'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_004_session02_run02_2018-04-06-09_35_41_subdata.log'
sub = 'sub-004'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_004_session02_run02_2018-04-06-09_35_41_subdata.log'
sub = 'sub-004'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_003_session02_2018-02-14-19_16_59_subdata.log'
sub = 'sub-003'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)



In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_023_session02_run03_2018-12-13-18_19_05_subdata.log'
sub = 'sub-023'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)



In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_023_session02_run02_2018-12-13-18_02_35_subdata.log'
sub = 'sub-023'
ses = 'ses-2'
run='run-1'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_020_session01_run01_2018-06-28-19_28_22_subdata.log'
sub = 'sub-020'
ses = 'ses-1'
run='run-1'
setup_onsets(z, sub, ses, 'training', run)



In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_020_session01_run02_2018-06-28-19_42_38_subdata.log'
sub = 'sub-020'
ses = 'ses-1'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)


In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_004_session01_2018-03-06-19_44_50_subdata.log'
sub = 'sub-004'
ses = 'ses-1'
run='run-1'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_004_session01_2018-03-06-20_00_17_subdata.log'
sub = 'sub-004'
ses = 'ses-1'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_002_session01_2017-10-27-17_58_14_subdata.log'
sub = 'sub-002'
ses = 'ses-1'
run='run-1'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_002_session01_2017-10-27-18_09_48_subdata.log'
sub = 'sub-002'
ses = 'ses-1'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_002_session02_2017-11-10-17_50_29_subdata.log'
sub = 'sub-002'
ses = 'ses-2'
run='run-1'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
z = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_002_session02_2017-11-10-18_01_29_subdata.log'
sub = 'sub-002'
ses = 'ses-2'
run='run-2'
setup_onsets(z, sub, ses, 'training', run)

In [ ]:
runnums = ['run-1','run-2','run-1','run-2']
tasks = ['training','training','pe','pe']

for sub, data in msms.items():
    num = sub.split('-')[-1]
    print(num)
    for sess, lists in data.items():
        print(sess)
        if len(lists) > 0:
            logs = glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets','bro_%s_*.log'%num))
            if len(logs) == 4:
                ordpaths = sorted(logs)
                print('%s %s has 4'%(sub,ses))
                for i,z in enumerate(ordpaths):
                    if os.path.getsize(z) > 8000:
                        print('%s %s here'%(sub,ses))
                        setup_onsets(z, sub, ses, tasks[i], runnums[i])
            
            
            
            
            
            

In [ ]:
runnums = ['run-1','run-2','run-1','run-2']
tasks = ['training','training','pe','pe']
#sub-037_ses-1_task-training_run-1_milkshake_cue.txt
still_missing = {}
for sub, sess in keep.items():
    print(sub)
    for ses, paths in sess.items():
        print(ses)
        if len(paths) == 4:
            ordpaths = sorted(paths)
            print('%s %s has 4'%(sub,ses))
            for i,z in enumerate(ordpaths):
                if os.path.exists(z):
                    print('%s %s here'%(sub,ses))
                    setup_onsets(z, sub, ses, tasks[i], runnums[i])
        elif len(paths) == 0:
            still_missing[sub] = sess
            continue
        else:
            print('%s %s has %s'%(sub,ses, len(paths)))
            num = sub.split('-')[-1]
            check4more = glob.glob('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_%s_*.log'%num)
            print('found %s for %s'%(len(check4more),sub))
            for z in paths:
                if os.path.exists(z):
                    print('%s %s here'%(sub,ses))
                    
                    
                    
                    
                    
                    

In [ ]:
subs = ['sub-003','sub-004','sub-007','sub-008','sub-009','sub-010','sub-011','sub-012','sub-014','sub-015','sub-016','sub-017','sub-018']

In [ ]:
runs = ['run-1','run-2']
for sub in subs:
    num = sub.split('-')[-1]
    logs = glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets','bro_%s*.log'%num))
    if len(logs) > 0:
        print('print found %s'%sub)
        ses = {'ses1':glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets','bro_%s_session01_*_subdata.log'%num)),
              'ses2': glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets','bro_%s_session02_*_subdata.log'%num))}
        for sess, data in ses.items():
            if len(data) == 4:
                sortdata = sorted(data)
                for i, item in enumerate(sortdata):
                    if i < 2:
                        print(runs[i])
                        setup_onsets(z, sub, sess, 'training', runs[i])
    
    
    
    
    
    
    

In [ ]:
bro_onset_files = sorted(glob.glob('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/*log'))


In [ ]:
keep = {}
for key, value in missing_ons.items():
    print(key)
    num = key.split('-')[1]
    sesnum = value['ses'].split('-')[1]
    print(sesnum)
    x = value['ses']
    keep[key] = {x:[]}
    tmp = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_%s_session0%s_*_subdata.log'%(num, sesnum)
    for log in glob.glob(tmp):
        print(os.path.getsize(log))
        if os.path.getsize(log) > 8000: 
            keep[key][x].append(log)
        elif os.path.getsize(log) > 6000:
            keep[key][x].append(log)
        else:
            print('empty')
    
    
    
    
    
    

In [ ]:
runnums = ['run-1','run-2','run-1','run-2']
tasks = ['training','training','pe','pe']
#sub-037_ses-1_task-training_run-1_milkshake_cue.txt
still_missing = {}
for sub, sess in keep.items():
    print(sub)
    for ses, paths in sess.items():
        print(ses)
        if len(paths) == 4:
            ordpaths = sorted(paths)
            print('%s %s has 4'%(sub,ses))
            for i,z in enumerate(ordpaths):
                if os.path.exists(z):
                    print('%s %s here'%(sub,ses))
                    setup_onsets(z, sub, ses, tasks[i], runnums[i])
        elif len(paths) == 0:
            still_missing[sub] = sess
            continue
        else:
            print('%s %s has %s'%(sub,ses, len(paths)))
            num = sub.split('-')[-1]
            check4more = glob.glob('/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets/bro_%s_*.log'%num)
            print('found %s for %s'%(len(check4more),sub))
            for z in paths:
                if os.path.exists(z):
                    print('%s %s here'%(sub,ses))
                    
                    
                    
                    
                    
                    

In [ ]:
task_id = ['training','pe']
run_num = ['run01','run02','run01','run02']
log = {}
for sub, sessions in keep.items():
    log[sub]={}
    for ses, fils in sessions.items():
        log[sub][ses]={}
        if len(fils) > 0:
            fils = sorted(fils)
            for i,f in enumerate(fils):
                if '_run0' in f:
                    print(sub)
                    print(f)
                    run = f.split('/')[-1].split('_')[3]
                    print(run)
                    if run == 'run01' or run=='run02':
                        setup_onsets(f,sub, ses, task_id[0], run)
                    if run == 'run03' or run=='run04':
                        setup_onsets(f,sub, ses, task_id[1], run)
                log[sub][ses][f]=[]
                if i <= 1:
                    setup_onsets(f,sub, ses, task_id[0], run_num[i])
                    log[sub][ses][f].append(task_id[0])
                    log[sub][ses][f].append(run_num[i])
                else:
                    setup_onsets(f,sub, ses, task_id[1], run_num[i])
                    log[sub][ses][f].append(task_id[1])
                    log[sub][ses][f].append(run_num[i])
            

In [ ]:
adillyofapickle('/projects/niblab/experiments/bromocriptine/data/onsets/tmp',log, 'onset_log')

## Checking the onsets
- sub-036_ses-2_task-training_run-1_h2O_cue.txt 
- sub-036_ses-2_task-training_run-1_h2O_delivery.txt 
- sub-036_ses-2_task-training_run-1_milkshake_cue.txt
- sub-036_ses-2_task-training_run-1_milkshake_delivery.txt
- sub-036_ses-2_task-training_run-1_rinse.txt  


In [35]:
bropath = '/projects/niblab/experiments/bromocriptine/data/onsets/bro_onsets'


In [36]:
check_onsets2 = {'ses-1':
                {'run-1':{'h2O_cue':{},'h2O_delivery':{},'milkshake_cue':{},'milkshake_delivery':{},'rinse':{}},
               'run-2':{'h2O_cue':{},'h2O_delivery':{},'milkshake_cue':{},'milkshake_delivery':{},'rinse':{}}},
                'ses-2':
                {'run-1':{'h2O_cue':{},'h2O_delivery':{},'milkshake_cue':{},'milkshake_delivery':{},'rinse':{}},
               'run-2':{'h2O_cue':{},'h2O_delivery':{},'milkshake_cue':{},'milkshake_delivery':{},'rinse':{}}}
               }

In [37]:
path1 = '/projects/niblab/experiments/bromocriptine/data/onsets/tmp'
path2 = '/projects/niblab/experiments/bromocriptine/data/onsets/output_onsets'

In [41]:
df1 = pd.merge(cvs['training1'], cvs['training2'], left_on='training1', right_on='training2')
df = pd.merge(df1, cvs['anat'], left_on='training1', right_on='anat')
df.to_csv(os.path.join(out_path,'complete_subs.csv'), index=False, header = False)  

In [42]:
subs = df['training1']
events = ['h2O_cue','h2O_delivery','milkshake_cue','milkshake_delivery','rinse']
runs = ['run-1','run-2']
sess = ['ses-1','ses-2']
# sub-030_ses-1_task-training_run-2_milkshake_delivery.txt

In [43]:
for event in events:
    for ses in sess:
        for run in runs:
            for sub in subs:
                if os.path.exists(os.path.join(path1,'%s_%s_task-training_%s_milkshake_delivery.txt'%(sub,ses,run))):
                    for event in events:
                        print(event)
                        f = os.path.join(path1,'%s_%s_task-training_%s_%s.txt'%(sub,ses,run,event))
                        df = pd.read_csv(f, sep = '\t', names = [sub,'dur','para'])
                        check_onsets2[ses][run][event][sub]=df[sub]
                elif os.path.exists(os.path.join(path2,'%s_%s_task-training_%s_milkshake_delivery.txt'%(sub,ses,run))):
                    for event in events:
                        print(event)
                        f = os.path.join(path2,'%s_%s_task-training_%s_%s.txt'%(sub,ses,run,event))
                        df = pd.read_csv(f, sep = '\t', names = [sub,'dur','para'])
                        check_onsets2[ses][run][event][sub]=df[sub]

h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milks

milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_del

h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milks

milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_del

milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h

h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_del

h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_delivery
rinse
h2O_cue
h2O_delivery
milkshake_cue
milkshake_del

In [44]:
adillyofapickle('/projects/niblab/experiments/bromocriptine/data/onsets/tmp',check_onsets2, 'tmp_onset_df')

already have tmp


In [45]:
from_task = {'ses-1':{'run-1':[],'run-2':[]},
            'ses-2':{'run-1':[],'run-2':[]}}
for x in glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/from_task','ses-*_run-*.csv')):
    ses = x.split('/')[-1].split('_')[0]
    run = x.split('/')[-1].split('_')[1].split('.')[0]
    print(run)
    if run == 'run-1':
        print(ses)
        from_task[ses]['run-1'].append(pd.read_csv(x))
    else:
        from_task[ses]['run-2'].append(pd.read_csv(x))

    
    
    

run-1
ses-1
run-1
ses-2
run-2
run-2


In [57]:
check_onsets2['ses-1']['run-1']['rinse']

{'sub-003': 0      16.3482
 1      32.3418
 2      48.3346
 3      60.3504
 4      76.3437
 5      91.3510
 6     106.3581
 7     122.3517
 8     135.3532
 9     153.3519
 10    165.3551
 11    184.3564
 12    196.3555
 13    209.3573
 14    223.3451
 15    234.3582
 16    247.3599
 17    265.3419
 18    279.3464
 19    292.3482
 20    307.3554
 21    322.3460
 22    341.3473
 23    353.3463
 24    366.3482
 25    381.3554
 26    393.3545
 27    410.3504
 28    423.3523
 29    435.3513
 30    453.3499
 31    465.3490
 Name: sub-003, dtype: float64,
 'sub-004': 0      16.3756
 1      35.3776
 2      49.3824
 3      63.3701
 4      77.3749
 5      91.3797
 6     107.3740
 7     120.3757
 8     133.3774
 9     147.3820
 10    162.3728
 11    175.3752
 12    193.3741
 13    205.3732
 14    217.3891
 15    230.3748
 16    242.3745
 17    253.3884
 18    265.3880
 19    281.3825
 20    294.3680
 21    312.3676
 22    327.3756
 23    341.3811
 24    356.3723
 25    375.3748
 26    394.3774
 2

In [46]:
events = ['h2O_cue','h2O_delivery','milkshake_cue','milkshake_delivery','rinse']


In [47]:
# water is 0
# milk is 1
bad = {'ses-1':{'run-1':[],'run-2':[]},
      'ses-2':{'run-1':[],'run-2':[]}}
flavs = [0,1]
log_bool2 = {}
for ses, x in check_onsets2.items():
    log_bool2[ses]={}
    for run, y in x.items():
        log_bool2[ses][run]={}
        for task, z in y.items():
            if task == 'h2O_cue':
                print('starting %s %s %s'%(ses,run,task))
                tru = from_task[ses][run][0][from_task[ses][run][0]['cond'] == 0]['cue'].reset_index(drop=True)+8
                df = pd.DataFrame.from_dict(check_onsets2[ses][run][task])
                test = df.sub(tru, axis = 0)
                mu = test.mean(axis=0)
                bad[ses][run].append([mu[abs(mu) > 1],task])
            if task == 'milkshake_cue':
                print('starting %s %s %s'%(ses,run,task))
                tru = from_task[ses][run][0][from_task[ses][run][0]['cond'] == 1]['cue'].reset_index(drop=True)+8
                df = pd.DataFrame.from_dict(check_onsets2[ses][run][task])
                test = df.sub(tru, axis = 0)
                mu = test.mean(axis=0)
                bad[ses][run].append([mu[abs(mu) > 1],task])
            if task == 'h2O_delivery':
                print('starting %s %s %s'%(ses,run,task))
                tru = from_task[ses][run][0][from_task[ses][run][0]['cond'] == 0]['taste'].reset_index(drop=True)+8
                df = pd.DataFrame.from_dict(check_onsets2[ses][run][task])
                test = df.sub(tru, axis = 0)
                mu = test.mean(axis=0)
                bad[ses][run].append([mu[abs(mu) > 1],task])
            if task == 'milkshake_delivery':
                print('starting %s %s %s'%(ses,run,task))
                tru = from_task[ses][run][0][from_task[ses][run][0]['cond'] == 1]['taste'].reset_index(drop=True)+8
                df = pd.DataFrame.from_dict(check_onsets2[ses][run][task])
                test = df.sub(tru, axis = 0)
                mu = test.mean(axis=0)
                bad[ses][run].append([mu[abs(mu) > 1],task])
            if task == 'rinse':
                print('starting %s %s %s'%(ses,run,task))
                tru = from_task[ses][run][0]['rinse'].reset_index(drop=True)+8
                df = pd.DataFrame.from_dict(check_onsets2[ses][run][task])
                test = df.sub(tru, axis = 0)
                mu = test.mean(axis=0)
                bad[ses][run].append([mu[abs(mu) > 1],task])
            



starting ses-1 run-1 h2O_cue
starting ses-1 run-1 h2O_delivery
starting ses-1 run-1 milkshake_cue
starting ses-1 run-1 milkshake_delivery
starting ses-1 run-1 rinse
starting ses-1 run-2 h2O_cue
starting ses-1 run-2 h2O_delivery
starting ses-1 run-2 milkshake_cue
starting ses-1 run-2 milkshake_delivery
starting ses-1 run-2 rinse
starting ses-2 run-1 h2O_cue
starting ses-2 run-1 h2O_delivery
starting ses-2 run-1 milkshake_cue
starting ses-2 run-1 milkshake_delivery
starting ses-2 run-1 rinse
starting ses-2 run-2 h2O_cue
starting ses-2 run-2 h2O_delivery
starting ses-2 run-2 milkshake_cue
starting ses-2 run-2 milkshake_delivery
starting ses-2 run-2 rinse


In [67]:
bad['ses-1']['run-1']

[[sub-004   -22.509719
  dtype: float64,
  'h2O_cue'],
 [sub-004   -22.49775
  dtype: float64,
  'h2O_delivery'],
 [sub-004    13.7457
  dtype: float64,
  'milkshake_cue'],
 [sub-004    13.750844
  dtype: float64,
  'milkshake_delivery'],
 [sub-004    -4.373453
  sub-013   -16.000000
  dtype: float64,
  'rinse']]

In [49]:
adillyofapickle('/projects/niblab/experiments/bromocriptine/data/onsets/tmp',bad, 'incorrect_onsets_df2')

already have tmp


In [ ]:
inccorect = onetoughjar('/projects/niblab/experiments/bromocriptine/data/onsets/tmp/tmp/incorrect_onsets_df2_06-02-2022_04-55-28')
fd_fail =  onetoughjar('/projects/niblab/scripts/BRO/NOTES/tmp/df_fail_06-02-2022_04-45-02')
ms = onetoughjar('/projects/niblab/scripts/BRO/NOTES/tmp/missingprep_summary_06-02-2022_04-03-45')


# Cleaning summary
## Skip (for now)
| sub          | ses         |  run      | reason     |
| -----------  | ----------- |-----------|----------- |
| sub-004      | ses-1       |  run-1    | wrong onset, FD fail|
| sub-004      | ses-1       |  run-2    | wrong onset|
| sub-004      | ses-2       |  run-1    | FD fail|
| sub-050      | ses-1       |  run-2    | wrong onset|
| sub-023      | ses-2       |  run-1    | wrong onset|
| sub-023      | ses-2       |  run-2    | wrong onset|
| sub-018      | ses-2       |  run-2    | wrong onset|
| sub-041      | ses-2       |  run-1    | wrong onset|
| sub-018      | ses-1       |  run-2    | FD fail    |
| sub-018      | ses-2       |  run-1    | FD fail    |
| sub-018      | ses-2       |  run-1    | FD fail    |
| sub-049      | ses-1       |  run-1    | FD fail    |
| sub-056      | ses-2       |  run-1    | FD fail    |
| sub-056      | ses-2       |  run-2    | FD fail    |
| sub-006      | all       |  all    | fmriprep fail    |
| sub-031      | all       |  all    | fmriprep fail    |
| sub-048      | all       |  all    | fmriprep fail    |
| sub-054      | all       |  all    | fmriprep fail    |
| sub-055      | all       |  all    | fmriprep fail    |
| sub-042      | all       |  all    | bad onsets    |


In [ ]:
bad = ['sub-004','sub-050','sub-023','sub-018','sub-041','sub-042','sub-049','sub-056','sub-006','sub-031','sub-048','sub-054','sub-055']

## Moved the weird onsets to a new folder

In [ ]:
san = ['high', 'low']
for ses, data in log_bool2.items():
    print(ses)
    for run, dat in data.items():
        print(run)
        for task, da in dat.items():
            print(task)
            for it in da:
                for i,sub in enumerate(list(it.columns)):
                    print('%s'%san[i])
                    print(sub)
                    tomove = glob.glob(os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/tmp','%s_%s_task-training_%s_*.txt'%(sub,ses,run)))
                    for item in tomove:
                        fil = item.split('/')[-1]
                        dest = os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/tmp/remove','%s'%fil)
                        shutil.move(item,dest)
                
                
                
                

## Notes
Number of volumns in untrimmed data is 243, the onsets should start at ~8 sec if using untrimmed data

In [58]:
def create_fsf(datadir, sub, ses, task, run):
    inputs = (sub,ses,task,run)
    outdir = '/projects/niblab/scripts/BRO/feat_fsf/level1/fsf_files'
    oot = '/projects/niblab/data/BRO/level1'
    #####################################
    error_file=os.path.join(outdir,'%s_ses-%s_task-%s_run-%s_error_file.txt'%inputs)
    error=open(error_file,'w')
    #####################################
    repl_dict = {}
    subpath = os.path.join(datadir,'%s'%sub,'ses-%s'%ses,'func','niftis')
    #####################################
    funcrun=os.path.join(subpath,'%s_ses-%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz'%inputs)
    if os.path.exists(funcrun):
        repl_dict.update({'FUNCTIONAL':funcrun})
    else:
        print('%s no func found'%sub)
        error.write('%s no func found\n'%sub)
        pass
    #####################################
    confounds=os.path.join(datadir,'%s'%sub,'ses-%s'%ses,'func','confounds','%s_ses-%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt'%inputs)
    repl_dict.update({'CONFOUND':confounds})
    #####################################
    ntmpts=check_output(['fslnvols',funcrun], universal_newlines=True)
    repl_dict.update({'VOL':ntmpts.strip()})
    #####################################
    output=os.path.join(oot,'%s_ses-%s_%s_run-%s.feat'%inputs)
    repl_dict.update({'OUTPUT':output})
    #####################################
    onsets=['h2O_cue', 'h2O_delivery', 'milkshake_cue', 'milkshake_delivery', 'rinse']
    repl = ['H2O_CUE','H2O_DELIVERY','MILKSHAKE_CUE','MILKSHAKE_DELIVERY','RINSE']
    for i,onset in enumerate(onsets):
        new_inputs = inputs+(onset,)
        onsfile1 = '%s_ses-%s_task-%s_run-%s_%s.txt'%(new_inputs)
#         onsfile2 = '%s_ses-%s_task-%s_run0%s_%s.txt'%(new_inputs)
        value1 = os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/output_onsets/',onsfile1)
#         value2 = os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/tmp/',onsfile2)
        value3 = os.path.join('/projects/niblab/experiments/bromocriptine/data/onsets/tmp/',onsfile1)
        
        if os.path.exists(value1):
            repl_dict[repl[i]]=value1
#         elif os.path.exists(value2):
#             repl_dict[repl[i]]=value2
#             if onset == 'rinse':
#                 tst = pd.read_csv(value2)
#                 if tst.shape[0] == 0:
#                     print('fix %s %s %s rinse'%(sub, ses, run))
#                     error.write('%s is missing rinse \n'%(sub))
#                     pass
        elif os.path.exists(value3):
            repl_dict[repl[i]]=value3
            if onset == 'rinse':
                tst = pd.read_csv(value3)
                if tst.shape[0] == 0:
                    print('fix %s %s %s rinse'%(sub, ses, run))
                    error.write('%s is missing rinse \n'%(sub))
                    pass
        else:
            print('%s %s %s no onsets found'%(sub,ses,run))
            error.write('%s %s %s no onsets found\n'%(sub,ses,run))
            pass
    #####################################
    mocos = ['moco0_file','moco1_file','moco2_file','moco3_file','moco4_file','moco5_file']
    for moco in mocos:
        m1 = moco.split('_')[0]
        new2_inputs = inputs+(m1,)
        v1=os.path.join('/projects/niblab/experiments/bromocriptine/data/preprocessed/%s/ses-%s/func/motion_parameters/'%(sub,ses))
        value = os.path.join(v1,'%s_ses-%s_task-%s_run-%s_%s.txt'%new2_inputs)
        repl_dict[moco]=value
    #####################################
    for key, value in repl_dict.items():
        if key in onsets:
            if os.path.exists(value)== False:
                error.write('%s is missing %s %s \n'%(sub,key,value))
    #####################################
    subfile=os.path.join(outdir,'%s_ses-%s_task-%s_run-%s.fsf'%inputs)
    with open(os.path.join('/projects/niblab/scripts/BRO/feat_fsf/level1/templates','%s_design1_ges.fsf'%(task)),'r') as infile:
        tempfsf=infile.read()
        for key in repl_dict:
            tempfsf = tempfsf.replace(key, repl_dict[key])
            with open(subfile,'w') as outfile:
                outfile.write(tempfsf)
    name = '%s_ses-%s_%s_run-%s'%inputs
    adillyofapickle(outdir,repl_dict, name)
    error.close()

    

## Find prepro that haven't been run yet

something still wrong with sub-003, 

In [60]:
subs = ['sub-003','sub-007','sub-008','sub-009','sub-010','sub-011','sub-012','sub-014','sub-015','sub-016',
'sub-017']




In [61]:
datadir = '/projects/niblab/experiments/bromocriptine/data/preprocessed'
sess = ['1','2']
runs = ['1','2']
task = 'training'

for ses in sess:
    for run in runs:
        for sub in subs:
            create_fsf(datadir, sub, ses, task, run)

already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp
already have tmp


In [ ]:
org = '/projects/niblab/experiments/bromocriptine/data/preprocessed'
to_run = []
rund = []
skip = []
for x in glob.glob(os.path.join(org,'*')):
    sub = x.split('/')[-1]
    if sub in bad:
        print('%s is bad skip'%sub)
        skip.append(sub)
    elif sub in done:
        print('%s already feated'%sub)
        rund.append(sub)
    else:
        print('%s need to feat'%sub)
        to_run.append(sub)
        
        
summary_dict = {'2run':to_run, 'ran':rund, 'skip':skip}        
adillyofapickle('/projects/niblab/data/BRO/level1/NOTES',summary_dict, 'who2run') 

In [ ]:
summary_dict.keys()

In [ ]:
datadir = '/projects/niblab/experiments/bromocriptine/data/preprocessed'
sess = ['1','2']
runs = ['1','2']
task = 'training'
for ses in sess:
    print(ses)
    for run in runs:
        print(run)
        for sub in summary_dict['2run']:
            print(sub)
            create_fsf(datadir, sub, ses, task, run)

In [ ]:
done = glob.glob(os.path.join('/projects/niblab/data/BRO/level1/sub-*'))

In [ ]:
dun = []
for y in done:
    x = y.split('/')[-1].split('.')[0]
    dun.append(x)

In [ ]:
fsfs ='/projects/niblab/scripts/BRO/feat_fsf/level1/fsf_files/*.fsf'
torun = {'ses-1':{'run-1':[],'run-2':[]},
        'ses-2':{'run-1':[],'run-2':[]}}
for fsf in glob.glob(fsfs):
    fs = fsf.split('/')[-1].split('.')[0]
    t = fs.split('_')
    fin = '%s_%s_training_%s'%(t[0],t[1],t[3])
    if fin in dun:
        print('-------')
    else:
        print(fin)     
        torun[t[1]][t[3]].append(t[0])

    


# Making a launcher

```
#!/bin/bash -l
#
#SBATCH -N 1
#SBATCH -c 1
#SBATCH -t TIME
###################
#SBATCH --mem-per-cpu MEMORY
## %A == SLURM_ARRAY_JOB_ID
## %a == SLURM_ARRAY_TASK_ID
#SBATCH -o OUTPUT
#SBATCH -e ERROR
# Run your executable


if [ ${SLURM_ARRAY_TASK_ID} -lt 10 ]; then
    id="00${SLURM_ARRAY_TASK_ID}"
elif [ ${SLURM_ARRAY_TASK_ID} -lt 100 ]; then
    id="0${SLURM_ARRAY_TASK_ID}"
else
    id="${SLURM_ARRAY_TASK_ID}"
fi



time /projects/niblab/modules/software/fsl/5.0.10/bin/feat SHELLFILE
```

## Notes
basic path = /projects/niblab/scripts/BRO/feat_fsf/level1/fsf_files  
basic file = sub-052_ses-2_task-training_run-1.fsf  
tmp file = /projects/niblab/scripts/BRO/job_files/temp.job

In [62]:
def create_job(pathtofsf, level, ses, run, mem, time, exitpaths):
    inputs = (level,ses,run)
    repl_dict = {}
    #####################################
    repl_dict.update({'TIME':time})
    repl_dict.update({'MEMORY':mem})
    #####################################
    output = os.path.join(exitpaths[0],'feat_lev%s_ses%s_run%s'%inputs+'%atraining_out.txt')
#     output = os.path.join(exitpaths[0],'feat_lev%s_ses%s_run%s_%atraining_out.txt'%inputs)
    repl_dict.update({'OUTPUT':output})
    #####################################
#     error = os.path.join(exitpaths[0],'feat_lev%s_ses%s_run%s_%atraining_err.txt'%inputs)
    error = os.path.join(exitpaths[0],'feat_lev%s_ses%s_run%s'%inputs+'%atraining_err.txt')
    repl_dict.update({'ERROR':error})
    shellfile = os.path.join(pathtofsf,'level%s/fsf_files/'%(level),'sub-${id}_ses-%s_task-training_run-%s.fsf'%(ses,run))
    repl_dict.update({'SHELLFILE':shellfile})
    #####################################
    subfile=os.path.join(os.getcwd(),'job_files','feat%s_ses-%s_run-%s.job'%inputs)
    with open(os.path.join('/projects/niblab/scripts/BRO/job_files','temp.job'),'r') as infile:
        tempfsf=infile.read()
        for key in repl_dict:
            tempfsf = tempfsf.replace(key, repl_dict[key])
            with open(subfile,'w') as outfile:
                outfile.write(tempfsf)
    name = 'feat%s_ses-%s_run-%s_job'%inputs
    adillyofapickle(pathtofsf,repl_dict, name)
    

    

In [ ]:
path2fsf = '/projects/niblab/scripts/BRO/feat_fsf/'
level = '1'
for sese, runs in torun.items():
    ses = sese.split('-')[-1]
    run = list(runs.keys())
    create_job(path2fsf, '%s'%level,'%s'%ses,'%s'%run[0].split('-')[-1],'8000','2:30:00', exitpaths)
    create_job(path2fsf, '%s'%level,'%s'%ses,'%s'%run[1].split('-')[-1],'8000','2:30:00', exitpaths)
    

sbatch --arrray=z,x,y job.job

## Command to launch jobs

In [64]:
runner = {'ses-1':{'run-1':subs,'run-2':subs},
         'ses-2':{'run-1':subs,'run-2':subs},}
    

In [63]:
def job_launcher(path2jobfile, level, dict_o_runs):
    for sese, runs in dict_o_runs.items():
        ses = sese.split('-')[-1]
        for run,subs in runs.items():
            ru = run.split('-')[-1]
            if os.path.exists(os.path.join(path2jobfile,'feat%s_ses-%s_run-%s.job'%(level,ses,ru))):
                y = len(subs)-1
                r = '%s,'*y
                cmd0 = 'sbatch --array='+r+'%s '+os.path.join(path2jobfile,'feat%s_ses-%s_run-%s.job'%(level,ses,ru))
                x = tuple(sub.split('-')[-1].strip("0") for sub in subs)
                cmd = cmd0%x
                print(cmd)
                cmd_split = cmd.split(' ')
#                 subprocess.run(cmd_split)
            else:
                print('need to make job file first!')


In [68]:
path2jobfile = '/projects/niblab/scripts/BRO/job_files'
job_launcher(path2jobfile , '1', runner)

sbatch --array=3,7,8,9,1,11,12,14,15,16,17 /projects/niblab/scripts/BRO/job_files/feat1_ses-1_run-1.job
sbatch --array=3,7,8,9,1,11,12,14,15,16,17 /projects/niblab/scripts/BRO/job_files/feat1_ses-1_run-2.job
sbatch --array=3,7,8,9,1,11,12,14,15,16,17 /projects/niblab/scripts/BRO/job_files/feat1_ses-2_run-1.job
sbatch --array=3,7,8,9,1,11,12,14,15,16,17 /projects/niblab/scripts/BRO/job_files/feat1_ses-2_run-2.job


## Check level 1 and registration work around 
https://mumfordbrainstats.tumblr.com/post/166054797696/feat-registration-workaround

In [ ]:
check_notes = {}
out_path = '/projects/niblab/data/BRO/'
for featdir in glob.glob(os.path.join(out_path, 'level1','sub*.feat')):
    check_notes[featdir] = []
    if len(glob.glob(os.path.join(featdir,'stats','cope*.nii.gz'))) == 10:
        print('complete')
        check_notes[featdir].append('complete')
        # make sure the reg_standard hasn't been created 
        if os.path.exists(os.path.join(featdir, 'reg_standard')):
            print('deleting reg standard')
            check_notes[featdir].append('deteled %s'%os.path.join(featdir, 'reg_standard'))
            shutil.rmtree(os.path.join(featdir, 'reg_standard'))
        for matfile in glob.glob(os.path.join(featdir,'reg','*.mat')):
            print(matfile) 
            if os.path.isfile(matfile):
                os.remove(matfile)
                check_notes[featdir].append('removed %s'%matfile)
            idmat = os.path.join('/projects/niblab/modules/software/fsl/5.0.10','etc','flirtsch','ident.mat')
            shutil.copyfile(idmat, matfile)
            check_notes[featdir].append('copied id mat from %s to %s'%(idmat, matfile))
    else:
        fail = os.path.join('/projects/niblab/data/BRO/level1/fail',featdir.split('/')[-1])
        check_notes[featdir].append('fail missing %s'%(10-len(glob.glob(os.path.join(featdir,'stats','cope*.nii.gz')))))
        shutil.move(featdir,fail)
            
                
            
            
adillyofapickle('/projects/niblab/data/BRO/level1/NOTES',check_notes, 'level1notes') 
            

In [ ]:
from IPython.display import HTML

HTML(filename="/projects/niblab/data/BRO/level1/fail/sub-035_ses-2_training_run-2.feat/report_log.html")


fails = {}
for x, y in check_notes.items():
#     print(y)
    for z in y:
        if 'fail missing ' in z:
            print(x)
            

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
bak_g = '/projects/niblab/modules/software/fsl/5.0.10/data/standard/tpl-MNI152NLin2009cAsym_res-01_T1w_brain.nii.gz'

In [ ]:
epi = '/projects/niblab/data/BRO/level1/sub-004_ses-2_training_run-2.feat/example_func.nii.gz'

In [ ]:
for brain in glob.glob(os.path.join('/projects/niblab/data/BRO/level1/sub-*_ses-*_training_run-*.feat/example_func.nii.gz')):
    sub = brain.split('/')[-2].split('_')[0]
    ses = brain.split('/')[-2].split('_')[1]
    run = brain.split('/')[-2].split('_')[3].split('.')[0]
#     print(run)
    reg_view(bak_g, brain, '%s %s %s'%(sub, ses, run))
                      
                      
                    

# Visualization

In [ ]:
import nibabel as nib

In [ ]:
epi = os.path.join('/projects/niblab/data/BRO/level1/sub-022_ses-1_training_run-1.feat','example_func.nii.gz')
epi_img = nib.load(epi)

In [ ]:
f = os.path.join(datadir,'sub-003','anat','highres.nii.gz')
img = nib.load(f)

In [ ]:
g = '/projects/niblab/experiments/bromocriptine/data/bids/bids/sub-001/ses-2/anat/sub-001_ses-2_T1w.nii.gz'
img2 = nib.load(g)

In [ ]:
import matplotlib

In [ ]:
from nilearn.plotting import view_img_on_surf, view_img
from nilearn import plotting


In [ ]:
bak_g = '/projects/niblab/modules/software/fsl/5.0.10/data/standard/tpl-MNI152NLin2009cAsym_res-01_T1w_brain.nii.gz'

In [ ]:
plotting.view_img(epi_img, bg_img = bak_g)
# plotting.plot_epi(epi_img)

In [ ]:
plotting.plot_anat(img, title="plot_anat")

In [ ]:
plotting.plot_anat(img2, title="plot_anat")

# Notes
~Need to look back into the skipping the registration in FSL (Jeannette)~


- 5/24 Submitted first batch of level 1, ses-1
- ~Next need to find the missing onsets and finish the level1s then check them~
- 5/25 Have found the weird onsets and checked the images, need to run the rest and correct the weird onsets
- ~May need to fmriprep missing images~ 
- fmripreping missing images see issue here and fix above: https://neurostars.org/t/fmriprep-error-positional-arguments-not-recognized-singularity/22691/2

